In [2]:
from vectorize import load, length
from tools import get_progressbar
import pandas
import numpy as np

/Users/dany/.local/share/virtualenvs/cuba-digital-lang-9Kd57kqM/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
_len_ = length('texts')
bar = get_progressbar(_len_, f' {_len_} scatter matrix computing ')

frame = dict([(f'x{i}', []) for i in range(96)])
vectors = []
vectors_to_text = {}
bar.start()
for i, tuple_ in enumerate(load('texts')):
    if len(tuple_[1]) > 0:
        vectors.append(np.array(tuple_[1]))
        vectors_to_text[tuple_[1]] = tuple_[0]

    for i, comp in enumerate(tuple_[1]):
        frame[f'x{i}'].append(comp)

    bar.update(i+1)
bar.finish()

[########################################] 162474 scatter matrix computing 100%


In [4]:
# df = pandas.DataFrame(frame)

# df.head(20)

In [5]:
# pandas.plotting.scatter_matrix(df)

In [22]:
from sklearn import preprocessing

vector = preprocessing.normalize(vectors)

In [19]:
from sklearn.cluster import DBSCAN
from sklearn import metrics

db = DBSCAN(eps=0.8, min_samples=10, metric='cosine', n_jobs=10).fit(vector)
# core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
# core_samples_mask[db.core_sample_indices_] = True

labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)
# print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
# print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
# print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
# print("Adjusted Rand Index: %0.3f" % metrics.adjusted_rand_score(labels_true, labels))
# print(
#     "Adjusted Mutual Information: %0.3f"
#     % metrics.adjusted_mutual_info_score(labels_true, labels)
# )
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(vector, labels))

Estimated number of clusters: 1
Estimated number of noise points: 0


ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

In [23]:
from sklearn.neighbors import DistanceMetric, kneighbors_graph
import matplotlib.pyplot as plt
dist = DistanceMetric.get_metric('manhattan')
matsim = dist.pairwise(vector)
minPits = 5
A = kneighbors_graph(vector, minPits, include_self=False).toarray()

seq = []
for i, s in enumerate(vector):
    for j in range(len(vector)):
        if A[i][j] != 0:
            seq.append(matsim[i][j])
    
seq.sort()
plt.plot(seq)
plt.show()

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

def plot_results(inertials):
    x, y = zip(*[inertia for inertia in inertials])
    plt.plot(x, y, 'ro-', markersize=8, lw=2)
    plt.grid(True)
    plt.xlabel('Num Clusters')
    plt.ylabel('Inertia')
    plt.show()


def select_clusters(loops, max_iterations, init_cluster, tolerance,
                    num_threads):
    # Read data set
    points = vectors

    inertia_clusters = list()

    for i in range(1, loops + 1, 1):
        # Object KMeans
        kmeans = KMeans(n_clusters=i, max_iter=max_iterations,
                        init=init_cluster, tol=tolerance, n_jobs=num_threads)

        # Calculate Kmeans
        kmeans.fit(points)

        # Obtain inertia
        inertia_clusters.append([i, kmeans.inertia_])

    plot_results(inertia_clusters)


In [ ]:

select_clusters(10, 1000, "k-means++", 0.001, 10)